In [38]:
# !pip install pyxirr
# !pip install numpy
# !pip install pandas

# if haven't downloaded the above packages, remove "#"

import numpy as np
import pandas as pd


excel_file1 = 'Challange Task Data.xlsm'  # access excel file, need to be in same folder as python file on desktop
df1 = pd.read_excel(excel_file1, "Canadian Stocks") # read tabs
df2 = pd.read_excel(excel_file1, "Canadian Stocks Benchmark")
df3 = pd.read_excel(excel_file1, "US Stocks")
df4 = pd.read_excel(excel_file1, "US Stocks Benchmark")

In [5]:
df1['Actual (Python)'] = - df1['Cashflow'] # actual cashflow, negative of cashflow
print(df1)

         Date        Value   Cashflow  Actual Cashflow  Unnamed: 4  \
0  2018-01-31  10849784.74  -11692.38         11692.38         NaN   
1  2018-02-28  10287646.81  -24322.38         24322.38         NaN   
2  2018-03-30  10275003.81  -46140.26         46140.26         NaN   
3  2018-04-30  10536100.39  -25996.38         25996.38         NaN   
4  2018-05-30  10851665.12  -25612.38         25612.38         NaN   
5  2018-06-30  11310824.25  -46415.97         46415.97         NaN   
6  2018-07-30  11438205.77  -26904.79         26904.79         NaN   
7  2018-08-30  11276744.62  -25805.29         25805.29         NaN   
8  2018-09-30  11021497.47  -46381.29         46381.29         NaN   
9  2018-10-30  10490587.08  -26904.79         26904.79         NaN   
10 2018-11-30  10705619.94  -26405.29         26405.29         NaN   
11 2018-12-30   9938734.37 -149992.55        149992.55         NaN   
12 2019-01-30  10802928.93  -12864.79         12864.79         NaN   
13 2019-02-28  11083

In [6]:
# sum up returns in each quarter for canadian stocks
# loop

qc = []  # Create an empty list to store our results.

i = 0  # Start counting at 0.

while i < len(df1['Actual Cashflow']):  # Keep going until we reach the end of the 'Actual Cashflow' data.
    qc.append(np.sum(df1['Actual Cashflow'][i:i+3]))  # Calculate the sum of the next three numbers and add it to our list 'qc'.
    i += 3  # Move to the next set of three numbers.

    
print(qc)  # Print the list of calculated sums.

[82155.02, 98024.73000000001, 99091.37, 203302.63, 89247.92, 107600.31, 125012.98999999999, 109038.87, 1066100.1099999999, 107390.11, 107162.42, 106950.25, 108322.84, 108157.37999999999, 109650.45999999999, 119309.22, 118284.17, 81018.22, 0.0, -43246.98999999999]


In [9]:
def create_quarterly_ac(df1, n=3):  # Define a function to create quarterly data from df1, canadian stocks data.
    
    i = 0  # Initialize 'i', the index.
    df_list_cad = []  # Create an empty list to store the new data.

    while i < len(df1.Date):  # Start a loop that goes through the 'Date' data.
        temp = []  # Create an empty temporary list to hold the data for one quarter.

        temp.append(df1.Date[i + (n - 1)])  # Add the 'Date' from the last month of each quarter.
        temp.append(df1.Value[i + (n - 1)])  # Add the 'Value' from the last month of each quarter.

        # Calculate the sum of 'Actual Cashflow' for the next 'n' elements, n here is 3.
        temp.append(df1['Actual Cashflow'][i:i + n].sum())

        df_list_cad.append(temp)  # Add this quarterly data to the list 'df_list_cad'.
        i += n  # Move to the next set of 'n' elements.

    # Create a new DataFrame with the quarterly data and specify column names.
    df_cad = pd.DataFrame(df_list_cad, columns=['Date', 'Value', 'Actual Cashflow'])

    return df_cad  # Return the resulting DataFrame.

df_cad = create_quarterly_ac(df1)  # Call the function to create quarterly data and store it in 'df_cad'.

print(df_cad) 


         Date        Value  Actual Cashflow
0  2018-03-30  10275003.81         82155.02
1  2018-06-30  11310824.25         98024.73
2  2018-09-30  11021497.47         99091.37
3  2018-12-30   9938734.37        203302.63
4  2019-03-30  11123149.13         89247.92
5  2019-06-30  11146899.63        107600.31
6  2019-09-30  11249173.25        125012.99
7  2019-12-30  11498725.48        109038.87
8  2020-03-30   7737016.08       1066100.11
9  2020-06-30   8103584.50        107390.11
10 2020-09-30   7740937.98        107162.42
11 2020-12-30   8678665.98        106950.25
12 2021-03-30   9893742.56        108322.84
13 2021-06-30  10784034.47        108157.38
14 2021-09-30  10976472.69        109650.46
15 2021-12-30  10570692.17        119309.22
16 2022-03-30  11375020.05        118284.17
17 2022-06-30  10840024.32         81018.22
18 2022-09-30  10575579.43             0.00
19 2022-12-30  11216853.22        -43246.99


In [10]:
df1['Date'] = pd.to_datetime(df1['Date']) # make the formate in date
df1.dtypes

Date                                        datetime64[ns]
Value                                              float64
Cashflow                                           float64
Actual Cashflow                                    float64
Unnamed: 4                                         float64
Unnamed: 5                                         float64
Date.1                                      datetime64[ns]
Value of Canadian Stocks (CAD)                     float64
Actual Cashflow of Canadian Stocks (CAD)           float64
Actual (Python)                                    float64
dtype: object

In [11]:
from pyxirr import xirr # import the xirr library we downloaded

def create_xirr_cad(df1, n=2):
    j = 0  # Initialize 'j', the index.
    xirr_ = []  # Create a list to store XIRR values.
    dates_ = pd.to_datetime(df_cad['Date'])  # Convert 'Date' column to datetime.
    dates2 = []  # Create a list to store the resulting dates.

    while j < len(df_cad['Actual Cashflow']) - (n - 1):
        temp = [
            -df_cad['Value'][j], # beginning value (negative)
            df_cad['Actual Cashflow'][j], # cashflow
            df_cad['Actual Cashflow'][j + (n - 1)], # cashflow
            df_cad['Value'][j + (n - 1)] # final value
        ]  # Create a list of cashflows for XIRR calculation.

        dates = [
            dates_[j],
            dates_[j],
            dates_[j + (n - 1)],
            dates_[j + (n - 1)]
        ]  # Create a list of dates for XIRR calculation.

        dates2.append(df_cad["Date"][j + (n - 1)])  # Append the resulting date to the 'dates2' list.

        # Calculate the date difference and adjust the XIRR result.
        date_diff = (dates_[j + n - 1] - dates_[j]) / pd.Timedelta(days=(n - 1))
        xirr_.append(xirr(dates, temp) * (date_diff) / 365)  # Calculate XIRR and adjust for time period.
        j += (n - 1)

    return xirr_, dates2  # Return the XIRR values and resulting dates.

xirr_, dates2 = create_xirr_cad(df1, n=2)  # Call the function with the DataFrame 'df1' and n=2.



In [17]:
xirr_cad = [i  for i in xirr_] 
xirr_cad
print(dates2, xirr_cad)

[Timestamp('2018-06-30 00:00:00'), Timestamp('2018-09-30 00:00:00'), Timestamp('2018-12-30 00:00:00'), Timestamp('2019-03-30 00:00:00'), Timestamp('2019-06-30 00:00:00'), Timestamp('2019-09-30 00:00:00'), Timestamp('2019-12-30 00:00:00'), Timestamp('2020-03-30 00:00:00'), Timestamp('2020-06-30 00:00:00'), Timestamp('2020-09-30 00:00:00'), Timestamp('2020-12-30 00:00:00'), Timestamp('2021-03-30 00:00:00'), Timestamp('2021-06-30 00:00:00'), Timestamp('2021-09-30 00:00:00'), Timestamp('2021-12-30 00:00:00'), Timestamp('2022-03-30 00:00:00'), Timestamp('2022-06-30 00:00:00'), Timestamp('2022-09-30 00:00:00'), Timestamp('2022-12-30 00:00:00')] [0.14211549727646144, -0.008123897170467057, -0.06412306054670788, 0.1906754578395883, 0.02059367865434872, 0.031728642733786665, 0.04640241619441761, -0.1605958379499811, 0.32257787553165185, -0.01801778848059333, 0.18876206475989982, 0.21448822477294757, 0.13353078720249095, 0.040674032435893605, -0.015877349976369273, 0.11594055065776987, -0.028527

In [13]:
# sum up returns in each quarter for canadian stocks benchmark

df2['Actual Cashflow'] = df2['Cashflow']
qc = []
i = 0
while i < len(df2['Actual Cashflow']):
    qc.append(np.sum(df2['Actual Cashflow'][i:i+3]))
    i += 3
print(qc)

[0.176, 0.178, 0.179, 0.18, 0.181, 0.193, 0.194, 0.207, 0.189, 0.199, 0.194, 0.197, 0.204, 0.196, 0.208, 0.221, 0.23, 0.236]


In [14]:
df_cad_benchmark = create_quarterly_ac(df2) # use the function we defined before
print(df_cad_benchmark)

         Date  Value  Actual Cashflow
0  2018-09-30  23.94            0.176
1  2018-12-30  23.00            0.178
2  2019-03-30  24.16            0.179
3  2019-06-30  24.65            0.180
4  2019-09-30  25.15            0.181
5  2019-12-30  25.56            0.193
6  2020-03-30  20.65            0.194
7  2020-06-30  23.55            0.207
8  2020-09-30  24.38            0.189
9  2020-12-30  26.07            0.199
10 2021-03-30  28.20            0.194
11 2021-06-30  30.53            0.197
12 2021-09-30  30.38            0.204
13 2021-12-30  32.52            0.196
14 2022-03-30  33.48            0.208
15 2022-06-30  29.01            0.221
16 2022-09-30  28.28            0.230
17 2022-12-30  29.59            0.236


In [21]:
def create_xirr_cad_benchmark(df2, n=2):
    j = 0  # Initialize'j', index.
    xirr_ = []  # Create a list to store XIRR values.
    dates_ = pd.to_datetime(df_cad_benchmark['Date'])  # Convert 'Date' column to datetime.
    dates3 = []  # Create a list to store the resulting dates.

    while j < len(df_cad_benchmark['Actual Cashflow']) - (n - 1):
        temp = [
            -df_cad_benchmark['Value'][j], # initial value, negatice
            df_cad_benchmark['Actual Cashflow'][j],
            df_cad_benchmark['Actual Cashflow'][j + (n - 1)],
            df_cad_benchmark['Value'][j + (n - 1)]
        ]  # Create a list of cashflows for XIRR calculation.

        dates = [
            dates_[j],
            dates_[j],
            dates_[j + (n - 1)],
            dates_[j + (n - 1)]
        ]  # Create a list of dates for XIRR calculation.

        dates3.append(df_cad_benchmark["Date"][j + (n - 1)])  # Append the resulting date to the 'dates3' list.

        # Calculate the date difference and adjust the XIRR result.
        date_diff = (dates_[j + n - 1] - dates_[j]) / pd.Timedelta(days=(n - 1))
        xirr_.append(xirr(dates, temp) * (date_diff) / 365)  # Calculate XIRR and adjust for the time period.
        j += (n - 1)

    return xirr_, dates3  # Return the XIRR values and resulting dates.

xirr_, dates3 = create_xirr_cad_benchmark(df2, n=2)  # Call the function with the DataFrame 'df2' and n=2.

xirr_cad_benchmark = [i for i in xirr_]  # Create a list 'xirr_cad_benchmark' containing XIRR values.

print(dates3) 


[Timestamp('2018-12-30 00:00:00'), Timestamp('2019-03-30 00:00:00'), Timestamp('2019-06-30 00:00:00'), Timestamp('2019-09-30 00:00:00'), Timestamp('2019-12-30 00:00:00'), Timestamp('2020-03-30 00:00:00'), Timestamp('2020-06-30 00:00:00'), Timestamp('2020-09-30 00:00:00'), Timestamp('2020-12-30 00:00:00'), Timestamp('2021-03-30 00:00:00'), Timestamp('2021-06-30 00:00:00'), Timestamp('2021-09-30 00:00:00'), Timestamp('2021-12-30 00:00:00'), Timestamp('2022-03-30 00:00:00'), Timestamp('2022-06-30 00:00:00'), Timestamp('2022-09-30 00:00:00'), Timestamp('2022-12-30 00:00:00')]


In [40]:
df3['Value'] = - df3['USD Value'] * df3['USD/CAD'] # actual cashflow in cad, negative of cashflow
df3['Actual Cashflow'] = - df3['Cashflow'] * df3['USD/CAD'] # actual cashflow in cad, negative of cashflow
df4['Value'] = - df4['USD Value'] * df4['USD/CAD'] # actual cashflow in cad, negative of cashflow
df4['Actual Cashflow'] = - df4['Cashflow'] * df4['USD/CAD'] # actual cashflow in cad, negative of cashflow
# print(df3)

# sum up returns in each quarter for us stocks 

qc = []
i = 0
while i < len(df3['Actual Cashflow']):
    qc.append(np.sum(df3['Actual Cashflow'][i:i+3]))
    i += 3
print(qc)

[417522.655435, -173581.15597199998, 406733.76405, 447936.71576000005, 446917.69864, 473789.71814, 441713.780195, 482080.72573999997, 442009.599665, 473806.23292, 411417.450985, 1444123.238045, 447360.88072, 492760.527075, 396522.194725, 279085.64448, 0.0, 859431.35025]


In [28]:
df_us = create_quarterly_ac(df3) # use the function we defined before
print(df_us)

         Date         Value  Actual Cashflow
0  2018-09-30 -7.681754e+07     4.175227e+05
1  2018-12-30 -7.826138e+07    -1.735812e+05
2  2019-03-30 -8.208639e+07     4.067338e+05
3  2019-06-30 -8.110675e+07     4.479367e+05
4  2019-09-30 -8.265833e+07     4.469177e+05
5  2019-12-30 -8.464274e+07     4.737897e+05
6  2020-03-30 -7.792480e+07     4.417138e+05
7  2020-06-30 -8.174697e+07     4.820807e+05
8  2020-09-30 -8.426174e+07     4.420096e+05
9  2020-12-30 -8.590682e+07     4.738062e+05
10 2021-03-30 -8.893742e+07     4.114175e+05
11 2021-06-30 -9.066293e+07     1.444123e+06
12 2021-09-30 -9.173691e+07     4.473609e+05
13 2021-12-30 -9.853936e+07     4.927605e+05
14 2022-03-30 -9.678756e+07     3.965222e+05
15 2022-06-30 -9.561682e+07     2.790856e+05
16 2022-09-30 -9.563899e+07     0.000000e+00
17 2022-12-30 -1.060913e+08     8.594314e+05


In [36]:
def create_xirr_us(df3, n=2):
    j = 0  # Initialize 'j', index.
    xirr_ = []  # Create a list to store XIRR values.
    dates_ = pd.to_datetime(df_us['Date'])  # Convert 'Date' column to datetime.
    dates4 = []  # Create a list to store the resulting dates.

    while j < len(df_us['Actual Cashflow']) - (n - 1):
        temp = [
            -df_us['Value'][j],  # Initial value (negative)
            df_us['Actual Cashflow'][j],
            df_us['Actual Cashflow'][j + (n - 1)],
            df_us['Value'][j + (n - 1)]
        ]  # Create a list of cashflows for XIRR calculation.

        dates = [
            dates_[j],
            dates_[j],
            dates_[j + (n - 1)],
            dates_[j + (n - 1)]
        ]  # Create a list of dates for XIRR calculation.

        dates4.append(df_us["Date"][j + (n - 1)])  # Append the resulting date to the 'dates4' list.

        # Calculate the date difference and adjust the XIRR result.
        date_diff = (dates_[j + n - 1] - dates_[j]) / pd.Timedelta(days=(n - 1))
        xirr_.append(xirr(dates, temp) * (date_diff) / 365)  # Calculate XIRR and adjust for the time period.
        j += (n - 1)

    return xirr_, dates4  # Return the XIRR values and resulting dates.

xirr_, dates4 = create_xirr_us(df3, n=2)  # Call the function with the DataFrame 'df3' and n=2.

xirr_us = [i for i in xirr_]  # Create a list 'xirr_us' containing XIRR values.

print(dates4) 

[Timestamp('2018-12-30 00:00:00'), Timestamp('2019-03-30 00:00:00'), Timestamp('2019-06-30 00:00:00'), Timestamp('2019-09-30 00:00:00'), Timestamp('2019-12-30 00:00:00'), Timestamp('2020-03-30 00:00:00'), Timestamp('2020-06-30 00:00:00'), Timestamp('2020-09-30 00:00:00'), Timestamp('2020-12-30 00:00:00'), Timestamp('2021-03-30 00:00:00'), Timestamp('2021-06-30 00:00:00'), Timestamp('2021-09-30 00:00:00'), Timestamp('2021-12-30 00:00:00'), Timestamp('2022-03-30 00:00:00'), Timestamp('2022-06-30 00:00:00'), Timestamp('2022-09-30 00:00:00'), Timestamp('2022-12-30 00:00:00')]


In [42]:
df_us_benchmark = create_quarterly_ac(df4) # use the function we defined before
print(df_us_benchmark)

         Date       Value  Actual Cashflow
0  2018-09-30 -375.697456        -1.709713
1  2018-12-30 -341.140800        -1.958775
2  2019-03-30 -377.534520        -1.647905
3  2019-06-30 -383.537000        -1.874488
4  2019-09-30 -392.923480        -1.832416
5  2019-12-30 -417.774280        -2.037860
6  2020-03-30 -361.494375        -1.971915
7  2020-06-30 -418.752880        -1.855028
8  2020-09-30 -445.906035        -1.782878
9  2020-12-30 -476.136180        -2.012130
10 2021-03-30 -498.384975        -1.607085
11 2021-06-30 -531.008430        -1.706928
12 2021-09-30 -543.076670        -1.807134
13 2021-12-30 -600.586920        -2.064928
14 2022-03-30 -564.550000        -1.707500
15 2022-06-30 -485.709375        -2.030388
16 2022-09-30 -493.801350        -2.206470
17 2022-12-30 -517.427790        -2.409693


In [43]:
def create_xirr_us_benchmark(df4, n=2):
    j = 0  # Initialize 'j', index.
    xirr_ = []  # Create a list to store XIRR values.
    dates_ = pd.to_datetime(df_us_benchmark['Date'])  # Convert 'Date' column to datetime.
    dates5 = []  # Create a list to store the resulting dates.

    while j < len(df_us_benchmark['Actual Cashflow']) - (n - 1):
        temp = [
            -df_us_benchmark['Value'][j],  # Initial value (negative)
            df_us_benchmark['Actual Cashflow'][j],
            df_us_benchmark['Actual Cashflow'][j + (n - 1)],
            df_us_benchmark['Value'][j + (n - 1)]
        ]  # Create a list of cashflows for XIRR calculation.

        dates = [
            dates_[j],
            dates_[j],
            dates_[j + (n - 1)],
            dates_[j + (n - 1)]
        ]  # Create a list of dates for XIRR calculation.

        dates4.append(df_us_benchmark["Date"][j + (n - 1)])  # Append the resulting date to the 'dates4' list.

        # Calculate the date difference and adjust the XIRR result.
        date_diff = (dates_[j + n - 1] - dates_[j]) / pd.Timedelta(days=(n - 1))
        xirr_.append(xirr(dates, temp) * (date_diff) / 365)  # Calculate XIRR and adjust for the time period.
        j += (n - 1)

    return xirr_, dates4  # Return the XIRR values and resulting dates.

xirr_, dates5 = create_xirr_us_benchmark(df3, n=2)  # Call the function with the DataFrame 'df3' and n=2.

xirr_us_benchmark = [i for i in xirr_]  # Create a list 'xirr_us_benchmark' containing XIRR values.

print(dates5) 


[Timestamp('2018-12-30 00:00:00'), Timestamp('2019-03-30 00:00:00'), Timestamp('2019-06-30 00:00:00'), Timestamp('2019-09-30 00:00:00'), Timestamp('2019-12-30 00:00:00'), Timestamp('2020-03-30 00:00:00'), Timestamp('2020-06-30 00:00:00'), Timestamp('2020-09-30 00:00:00'), Timestamp('2020-12-30 00:00:00'), Timestamp('2021-03-30 00:00:00'), Timestamp('2021-06-30 00:00:00'), Timestamp('2021-09-30 00:00:00'), Timestamp('2021-12-30 00:00:00'), Timestamp('2022-03-30 00:00:00'), Timestamp('2022-06-30 00:00:00'), Timestamp('2022-09-30 00:00:00'), Timestamp('2022-12-30 00:00:00'), Timestamp('2018-12-30 00:00:00'), Timestamp('2019-03-30 00:00:00'), Timestamp('2019-06-30 00:00:00'), Timestamp('2019-09-30 00:00:00'), Timestamp('2019-12-30 00:00:00'), Timestamp('2020-03-30 00:00:00'), Timestamp('2020-06-30 00:00:00'), Timestamp('2020-09-30 00:00:00'), Timestamp('2020-12-30 00:00:00'), Timestamp('2021-03-30 00:00:00'), Timestamp('2021-06-30 00:00:00'), Timestamp('2021-09-30 00:00:00'), Timestamp('20

In [18]:
result_cad = [0] + [a - b for a, b in zip(xirr_cad[1:], xirr_cad[:-1])]  # Calculate differences in xirr_cad

dates_for_cad_results = dates2[0:]  # Create a list of dates for results.

results_cad = [dates_for_cad_results, result_cad]  # Combine dates and result_cad.

final_cad = pd.DataFrame(results_cad).transpose()  # Create a DataFrame and transpose it to vertical.

final_cad.columns = ['Date', 'num']  # Rename columns in the DataFrame.

final_cad  # 'final_cad' now contains the results with 'Date' and 'num' columns.


,Date,num
0,2018-06-30,0
1,2018-09-30,-0.150239
2,2018-12-30,-0.055999
3,2019-03-30,0.254799
4,2019-06-30,-0.170082
5,2019-09-30,0.011135
6,2019-12-30,0.014674
7,2020-03-30,-0.206998
8,2020-06-30,0.483174
9,2020-09-30,-0.340596


In [53]:
result_cad_benchmark = [0] + [a - b for a, b in zip(xirr_cad_benchmark[1:], xirr_cad_benchmark[:-1])] 
result_cad_benchmark
dates_for_cad_benchmark_results = dates3[:len(result_cad)]
print(len(dates_for_cad_benchmark_results))
results_cad_benchmark = [dates_for_cad_benchmark_results, result_cad_benchmark]
final_cad_benchmark = pd.DataFrame(results_cad_benchmark).transpose()
final_cad_benchmark.columns = ['Date', 'num']
final_cad_benchmark

17


,Date,num
0,2018-12-30,0
1,2019-03-30,0.097293
2,2019-06-30,-0.036228
3,2019-09-30,-0.000241
4,2019-12-30,-0.004151
5,2020-03-30,-0.168818
6,2020-06-30,0.340158
7,2020-09-30,-0.147499
8,2020-12-30,0.040905
9,2021-03-30,0.015386


In [54]:
result_ = [0] + [a - b for a, b in zip(xirr_cad_benchmark[1:], xirr_cad_benchmark[:-1])] 
result_cad_benchmark
dates_for_cad_benchmark_results = dates3[:len(result_cad_benchmark)]
print(len(dates_for_cad_benchmark_results))
results_cad_benchmark = [dates_for_cad_benchmark_results, result_cad_benchmark]
final_cad_benchmark = pd.DataFrame(results_cad_benchmark).transpose()
final_cad_benchmark.columns = ['Date', 'num']
final_cad_benchmark

17


,Date,num
0,2018-12-30,0
1,2019-03-30,0.097293
2,2019-06-30,-0.036228
3,2019-09-30,-0.000241
4,2019-12-30,-0.004151
5,2020-03-30,-0.168818
6,2020-06-30,0.340158
7,2020-09-30,-0.147499
8,2020-12-30,0.040905
9,2021-03-30,0.015386


In [55]:
result_us = [0] + [a - b for a, b in zip(xirr_us[1:], xirr_us[:-1])]
result_us
dates_for_us_results = dates4[:len(result_us)]
print(len(dates_for_us_results))
results_us = [dates_for_us_results, result_us]
final_us = pd.DataFrame(results_us).transpose()
final_us.columns = ['Date', 'num']
final_us

17


,Date,num
0,2018-12-30,0
1,2019-03-30,0.03343
2,2019-06-30,-0.070846
3,2019-09-30,0.029662
4,2019-12-30,0.004899
5,2020-03-30,-0.091333
6,2020-06-30,0.117349
7,2020-09-30,-0.019157
8,2020-12-30,-0.011185
9,2021-03-30,0.017073


In [56]:
result_us_benchmark = [0] + [a - b for a, b in zip(xirr_us_benchmark[1:], xirr_us_benchmark[:-1])]
result_us_benchmark
dates_for_us_benchmark_results = dates5[:len(result_us_benchmark)]
print(len(dates_for_us_benchmark_results))
results_us_benchmark = [dates_for_us_benchmark_results, result_us_benchmark]
final_us_benchmark = pd.DataFrame(results_us_benchmark).transpose()
final_us_benchmark.columns = ['Date', 'num']
final_us_benchmark


17


,Date,num
0,2018-12-30,0
1,2019-03-30,0.213829
2,2019-06-30,-0.114642
3,2019-09-30,0.009784
4,2019-12-30,0.045873
5,2020-03-30,-0.185837
6,2020-06-30,0.32158
7,2020-09-30,-0.135358
8,2020-12-30,0.003561
9,2021-03-30,-0.026631


In [61]:
data_cad = {'Date': dates_for_cad_results, 'Column2': xirr_cad}
df_cad = pd.DataFrame(data_cad)

# Export the DataFrame to an Excel file
excel_file = 'canadian_stock_irr_quarter.xlsx' 
df_cad.to_excel(excel_file, index=False) 

data_cad_benchmark = {'Date': dates_for_cad_benchmark_results, 'Column3': xirr_cad_benchmark}
df_cad_benchmark = pd.DataFrame(data_cad_benchmark)

# Export the DataFrame to an Excel file
excel_file = 'canadian_stock_benchmark_irr_quarter.xlsx' 
df_cad_benchmark.to_excel(excel_file, index=False) 

data_us = {'Date': dates_for_us_results, 'Column2': xirr_us}
df_us = pd.DataFrame(data_us)

# Export the DataFrame to an Excel file
excel_file = 'us_stock_irr_quarter.xlsx' 
df_cad.to_excel(excel_file, index=False) 

data_us_benchmark = {'Date': dates_for_us_benchmark_results, 'Column2': xirr_us_benchmark}
df_us_benchmark = pd.DataFrame(data_us_benchmark)

# Export the DataFrame to an Excel file
excel_file = 'us_stock_benchmark_irr_quarter.xlsx' 
df_cad.to_excel(excel_file, index=False) 

In [62]:
df_merged1 = pd.merge(final_cad, final_cad_benchmark, how='outer',on=['Date']) # merging tables for cad
df_merged1['Date'] = pd.to_datetime(df_merged1['Date']).dt.date
print(df_merged1)
excel_file = 'canadian_irr_diff.xlsx' 
df_merged1.to_excel(excel_file, index=False) 

df_merged2 = pd.merge(final_us, final_us_benchmark, how='outer',on=['Date']) # merging tables for cad
df_merged2['Date'] = pd.to_datetime(df_merged2['Date']).dt.date
print(df_merged2)
excel_file = 'us_irr_diff.xlsx' 
df_merged2.to_excel(excel_file, index=False) 

          Date     num_x     num_y
0   2018-06-30         0       NaN
1   2018-09-30 -0.150239       NaN
2   2018-12-30 -0.055999         0
3   2019-03-30  0.254799  0.097293
4   2019-06-30 -0.170082 -0.036228
5   2019-09-30  0.011135 -0.000241
6   2019-12-30  0.014674 -0.004151
7   2020-03-30 -0.206998 -0.168818
8   2020-06-30  0.483174  0.340158
9   2020-09-30 -0.340596 -0.147499
10  2020-12-30   0.20678  0.040905
11  2021-03-30  0.025726  0.015386
12  2021-06-30 -0.080957 -0.000972
13  2021-09-30 -0.092857   -0.1037
14  2021-12-30 -0.056551  0.087077
15  2022-03-30  0.131818 -0.050456
16  2022-06-30 -0.144468 -0.146257
17  2022-09-30  0.011905  0.091707
18  2022-12-30  0.078169  0.079164
          Date     num_x     num_y
0   2018-12-30         0         0
1   2019-03-30   0.03343  0.213829
2   2019-06-30 -0.070846 -0.114642
3   2019-09-30  0.029662  0.009784
4   2019-12-30  0.004899  0.045873
5   2020-03-30 -0.091333 -0.185837
6   2020-06-30  0.117349   0.32158
7   2020-09-30 -0.01